In [25]:
merged_df_NEW = pd.read_csv('merged_df_NEW.csv')



import pandas as pd

# Load your dataset
# merged_df = pd.read_csv('path_to_your_dataset.csv')

# Display a summary of missing values
print(merged_df.isnull().sum())



Unnamed_0                0
ID                       0
group                    0
outcome                  0
age                      0
                        ..
religion                 0
marital_status          27
ethnicity                0
diagnosis                0
hospital_expire_flag     0
Length: 66, dtype: int64


## SINGLE IMPUTATION

In [11]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Assuming merged_df is your DataFrame
# merged_df = pd.read_csv('path_to_your_dataset.csv')

# Convert datetime columns to numeric (timestamp)
merged_df['admittime'] = pd.to_datetime(merged_df['admittime'])
merged_df['admittime'] = merged_df['admittime'].astype(int) / 10**9  # Convert datetime to seconds since epoch

# Columns to exclude from imputation
exclude_columns = ['deathtime']

# Separate columns to impute and columns to exclude
columns_to_impute = merged_df.drop(columns=exclude_columns).select_dtypes(include=[np.number]).columns
excluded_data = merged_df[exclude_columns]

# Initialize the Iterative Imputer
imputer = IterativeImputer(max_iter=10, random_state=0)

# Perform the imputation
imputed_data = imputer.fit_transform(merged_df[columns_to_impute])

# Convert the imputed data back to a DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=columns_to_impute)

# Add excluded columns back to the DataFrame
imputed_df = pd.concat([imputed_df, excluded_data.reset_index(drop=True)], axis=1)

# If you need to convert the datetime back to its original form
imputed_df['admittime'] = pd.to_datetime(imputed_df['admittime'] * 10**9)

# Display the imputed DataFrame
print(imputed_df)


      Unnamed: 0        ID  group  outcome   age  gendera        BMI  \
0            1.0  100213.0    1.0      0.0  74.0      2.0  26.814570   
1            2.0  100449.0    2.0      0.0  87.0      1.0  24.000000   
2            3.0  100571.0    1.0      0.0  67.0      1.0  32.284350   
3            4.0  100610.0    1.0      0.0  81.0      2.0  26.614580   
4            5.0  100660.0    1.0      0.0  75.0      1.0  29.315164   
...          ...       ...    ...      ...   ...      ...        ...   
1172      1173.0  199859.0    2.0      0.0  55.0      2.0  37.213436   
1173      1174.0  199861.0    2.0      0.0  89.0      2.0  27.403770   
1174      1175.0  199912.0    2.0      1.0  89.0      2.0  25.080360   
1175      1176.0  199925.0    2.0      0.0  86.0      1.0  20.415225   
1176      1177.0  199952.0    1.0      0.0  85.0      1.0  33.706539   

      hypertensive  atrialfibrillation  CHD with no MI  ...  Anion gap  \
0              1.0                 0.0             0.0  ...  

In [14]:
# Save the imputed DataFrame to a CSV file
# Save the imputed DataFrame to a CSV file in the current working directory
imputed_df.to_csv('imputed_dataset.csv', index=False)



## MULTIPLE IMPUTATION

In [29]:
import pandas as pd
import numpy as np
import miceforest as mf
import re

# Convert datetime columns to numeric (timestamp)
merged_df_NEW['admittime'] = pd.to_datetime(merged_df_NEW['admittime'])
merged_df_NEW['admittime'] = merged_df_NEW['admittime'].astype(int) / 10**9  # Convert datetime to seconds since epoch

# Clean column names to remove special characters
cleaned_columns = [re.sub(r'\W+', '_', col) for col in merged_df_NEW.columns]
merged_df_NEW.columns = cleaned_columns

# Check the column names
print(merged_df_NEW.columns)

# Columns to exclude from imputation
exclude_columns = ['survival_time', 'deathtime', 'LOS']

# Ensure exclude_columns exist in the DataFrame
exclude_columns = [col for col in exclude_columns if col in merged_df_NEW.columns]

# Separate columns to impute and columns to exclude
columns_to_impute = merged_df_NEW.drop(columns=exclude_columns).select_dtypes(include=[np.number]).columns
data_to_impute = merged_df_NEW[columns_to_impute]

# Initialize the KernelDataSet
kernel = mf.ImputationKernel(
    data=data_to_impute,
    save_all_iterations=True,
    random_state=0
)

# Perform multiple imputation with 5 imputations
kernel.mice(30)

# Extract the completed data from the first imputation
completed_data = kernel.complete_data(0)

# Combine the imputed data with the excluded columns
imputed_df_NEW = pd.concat([completed_data, merged_df_NEW[exclude_columns].reset_index(drop=True)], axis=1)

# If you need to convert the datetime back to its original form
imputed_df_NEW['admittime'] = pd.to_datetime(imputed_df_NEW['admittime'] * 10**9)

# Save the imputed DataFrame to a CSV file
imputed_df_NEW.to_csv('multiple_imputation_NEW.csv', index=False)




Index(['Unnamed_0', 'V1', 'ID', 'group', 'outcome', 'age', 'gendera', 'BMI',
       'hypertensive', 'atrialfibrillation', 'CHD_with_no_MI', 'diabetes',
       'deficiencyanemias', 'depression', 'Hyperlipemia', 'Renal_failure',
       'COPD', 'heart_rate', 'Systolic_blood_pressure',
       'Diastolic_blood_pressure', 'Respiratory_rate', 'temperature', 'SP_O2',
       'Urine_output', 'hematocrit', 'RBC', 'MCH', 'MCHC', 'MCV', 'RDW',
       'Leucocyte', 'Platelets', 'Neutrophils', 'Basophils', 'Lymphocyte',
       'PT', 'INR', 'NT_proBNP', 'Creatine_kinase', 'Creatinine',
       'Urea_nitrogen', 'glucose', 'Blood_potassium', 'Blood_sodium',
       'Blood_calcium', 'Chloride', 'Anion_gap', 'Magnesium_ion', 'PH',
       'Bicarbonate', 'Lactic_acid', 'PCO2', 'EF', 'subject_id', 'admittime',
       'dischtime', 'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'diagnosis', 'hospital_expi

In [32]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Example data
np.random.seed(0)
X = np.random.randn(100, 2)  # 100 samples, 2 predictors
y = X[:, 0] * 3 + X[:, 1] * 2 + np.random.randn(100)  # True relationship with some noise

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize KNN regressor with k=3
knn = KNeighborsRegressor(n_neighbors=3)

# Fit the model
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Predicted values:", y_pred)
print("True values:", y_test)


Mean Squared Error: 2.0854518609862387
Predicted values: [-3.39656071  0.04747127  3.53496476  8.75013829  2.28469933 -2.35298842
 -5.93260792 -2.06401683  8.08302764 -2.93982082 -2.74377608 -1.74395294
  0.98950967 -0.9228023   0.47497881 -2.55150005 -2.55150005 -3.65759875
 -2.53249913  2.34343668]
True values: [-3.73717324  0.22318728  4.74777781  8.10951534  2.83100725 -4.26883568
 -6.64197059 -0.11434078  9.42139441 -2.20757763 -3.05509498  0.97549574
 -0.81063972 -0.70351124  1.26090744 -4.16054858 -2.86870112 -6.80797112
 -3.54898048  4.35184529]
